In [1]:
import os
import re
import json
import UnityPy
from collections import defaultdict
from datetime import datetime

In [2]:
# game data folder location
urbek_data = r"H:\Steam\steamapps\common\Urbek\Urbek_Data\resources.assets"
modified_date = datetime.fromtimestamp(os.path.getmtime(fr"{urbek_data}")).strftime('%Y-%m-%d')
modified_date

'2022-10-22'

In [131]:
# utils
gray = lambda x: f'<span class="gray">{x}</span>'
gray_self = lambda x, codeName: str(x).replace(f"'{codeName}'", gray(f"'{codeName}'"))
plus_minus = lambda x: f'<span class="plus">+{x}</span>' if x > 0 else f'<span class="minus">{x}</span>' if x < 0 else '0'
more_less = lambda biggerThan, x: str(x) if biggerThan else f'<span class="less">{"" if str(x)[0] == "0" else "≤"}{x}</span>'
extracted = lambda x: f'<span class="extracted">{x}</span>'
tr = lambda arr: ''.join(f"<tr>{i}</tr>" for i in arr)
th = lambda arr: ''.join(f"<th>{i}</th>" for i in arr)
td = lambda arr: ''.join(f"<td>{i}</td>" for i in arr)
apos = lambda x: str(x).replace("'", "‛")
translate = lambda x, lang='en': f'<span class="translate" title=\'{apos(json.dumps(x, indent=2, ensure_ascii=False))}\'>{x[lang]}</span>'
option = lambda arr: ''.join(f"<option>{i}</option>" for i in arr)

In [132]:
# load data file from game installation
env = UnityPy.load(urbek_data)

rules = {}
translations = {}
for obj in env.objects:
    if obj.type.name != "TextAsset": continue
    data = obj.read()
    # download json files for browsing:
    os.makedirs("./data", exist_ok=True)
    with open(f"./data/{data.name}.json", "wb") as f:
        f.write(data.script)
    if data.name == "gameValues":
        for item in json.loads(data.text):
            translations[item["id"]] = item
    if re.match(r"reglasBioma_\d$", data.name) and \
       re.search(r"localVariables", data.text):
        rules[data.name] = json.loads(data.text)
del env, obj, data

# explore list of languages
languages = [i for i in translations["resources_comida"].keys() if i not in ['id', 'INFO']]
languages

['en', 'es', 'fr', 'zh-CN', 'pl', 'de', 'pt', 'ru', 'ja', 'it', 'ko']

In [133]:
labels = dict((k, v)
              for k, v in translations.items()
              if not re.search(r"text|desc|name|effect|t_|nr_", k))

# explore list of (non-empty) biomes
dict(sorted([(k, re.sub(r"<b>|</b>[\s\S]*", "", v["description"])) for k, v in rules.items()]))

{'reglasBioma_0': 'Temperate',
 'reglasBioma_1': 'Desert',
 'reglasBioma_2': 'Archipelago',
 'reglasBioma_3': 'Forest',
 'reglasBioma_4': 'Ruins'}

In [134]:
# choose a biome
biome = "reglasBioma_4"
data = rules[biome]
columns_resources = [item.split(',')[0] for item in data["resources"]]
columns_lv = [item.split(',')[0] for item in data["localVariables"]]

# parse buildings form the biome
buildings = []
updatesFromDict = defaultdict(list)
for building in data["construs"]:
    if up := building.get('updates'):
        for u in up:
            updatesFromDict[u].append(building['codeName'])
    
for building in sorted(data["construs"], key=lambda b: next((x['q'] for x in b.get('localVariables', []) if x['locVar'] == 'densidad'), 999)):
    codeName = building['codeName']
    label_all = labels.get(f'b_{codeName}', {})
    label = label_all.get('en', '???')
    category = building.get('category')
    category = f"{labels.get(f'bg_{category}')} ({gray(category)})" if category else ""
    distanceToRoad = building.get('distanceToRoad', '')
    if distanceToRoad == 3:
        distanceToRoad = gray(distanceToRoad)
    updates = gray_self(building.get('updates', ''), codeName)
    updatesFrom = gray_self(updatesFromDict.get(codeName, ''), codeName)
    buildingsNeeded = [more_less(p['biggerThan'], f"{p['q']} {p['building']} ⊙{p['radio'] if p['radio'] else '∞'}")
                       for p in building.get('buildingsNeeded')
                      ] if building.get('buildingsNeeded') else ''
    unblock = [f"{p['q']} {p.get('gloVar', p.get('nBuildings', '?'))}"
               for p in building.get('unblock')
              ] if building.get('unblock') else ''
    minModels = min([re.sub(r"^[^,]*,|,E?$", "", p) for p in building.get('models') if p
                    ]) if building.get('models') else ''
    if minModels == '1,1':
        minModels = gray(minModels)
    
    lvn = [next((more_less(p['biggerThan'], f"{p['q']}{'□ ' + str(p['threshold']).rjust(2) if 'threshold' in p else ''} ⊙{p['radio']}")
                 for p in building.get('localVariablesNeeded', [])
                 if p['localVariable'] == col), "")
           for col in columns_lv]
    lv = [next((plus_minus(p['q'])
                 for p in building.get('localVariables', [])
                 if p['locVar'] == col), "")
          for col in columns_lv]
    rn = [next((more_less(p['biggerThan'], f"{p['q']} ⊙{p['radio']}")
                for p in building.get('resourcesNeeded', [])
                if p['resource'] == col), "")
          for col in columns_resources]
    rx = [next((extracted(f"-{p['q']} ⊙{p['distance']}")
                for p in ([building.get('resourceExtraction')] if building.get('resourceExtraction') else [])
                if p['resource'] == col), "")
          for col in columns_resources]
    r = [next((plus_minus(p['q'])
               for p in building.get('produces', [])
               if p['rec'] == col), "")
         for col in columns_resources]
    
    lvnlv = [f"{a}{' '+b.rjust(3) if b else ''}" for a, b in zip(lvn, lv)]
    rnrxr = [f"{a}{' '+b.rjust(3) if b else ''}{' '+c.rjust(3) if c else ''}" for a, b, c in zip(rn, rx, r)]
    
    if any([updates, unblock, *rn, *rx, *r, *lvn, *lv]):
        buildings.append([translate(label_all),
                          distanceToRoad,
                          minModels,
                          *lvnlv,
                          *rnrxr,
                          codeName,
                          buildingsNeeded,
                          unblock,
                          updatesFrom,
                          updates,
                          category,
                         ])

# visualize buildings
columns = [f"Building name", "distanceToRoad", "smallest size",
           *[f"{translate(labels[f'lv_{c}'])}  ({gray(c)})" for c in columns_lv],
           *[f"{translate(labels[f'resources_{c}'])}  ({gray(c)})" for c in columns_resources],
           "codeName", "buildingsNeeded", "unblock", "updatesFrom", "updates", "category",
          ]

In [135]:
html = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Urberk buildings</title>
  <meta name="viewport"
        content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0">
  <style>
  body {
      margin: 0 0 60px;
      color: white;
      font: .9rem sans-serif;
      background: #252525;
      width: fit-content;
  }
  .intro {
      position: sticky;
      left: 0;
      width: 100vw;
  }
  table {
      position: relative;
      border-collapse: collapse;
  }
  @media (min-height: 600px) {
      thead {
          position: sticky;
          top: 0;
          z-index: 1;
      }
  }
  thead tr {
      height: 225px;
      background: black;
  }
  thead th {
      white-space: pre;
      vertical-align: bottom;
      text-align: left;
      padding: 5px;
  }
  thead tr th:nth-child(n+2):not(:nth-last-child(-n+6)) {
      transform-origin: bottom left;
      transform: translateX(20px) rotate(-60deg);
      max-width: 20px;
  }
  tr {
      background: #333;
  }
  tr:nth-child(2n) {
      background: #222;
  }
  tr:hover {
      background: black !important;
  }
  td {
      white-space: pre;
      border-left: 1px dotted #333;
      padding: 5px;
  }
  @media (min-width: 600px) {
      td:nth-child(1) {
          position: sticky;
          left: 0;
          background: inherit;
      }
  }
  td:nth-child(n+2) {
      text-align: right;
      font-family: monospace;
  }
  td:nth-last-child(-n+6) {
      text-align: left;
      max-width: 250px;
      overflow: hidden;
      white-space: nowrap;
      text-overflow: ellipsis;
      line-height: 1.2;
  }
  td:nth-last-child(-n+5):not(:last-child):hover {
      position: absolute;
      white-space: pre-wrap;
      background: black;
  }
  h1 { margin: 15px 14px; }
  p, dt { margin-left: 15px; }
  dd { font-family: monospace; }
  a { color: goldenrod; transition: text-decoration-color 200ms; }
  a:hover { text-decoration-color: orange; }
  .gray { color: #999; font-weight: normal; }
  .less { color: goldenrod; }
  .plus { color: #7f7; }
  .minus { color: #f66; }
  .extracted { color: #d6f; }
  </style>
</head>""" + f"""
<body>
  <div class="intro">
    <h1>
      List of buildings from
      <a href="https://store.steampowered.com/app/1411740/Urbek_City_Builder/">
        Urbek&nbsp;City&nbsp;Builder
      </a>
    </h1>
    <p>This is a fan-made page. Game data extracted with UnityPy, from <i>reglasBioma_*</i>
      TextAsset files inside <i>resources.assets</i> ({modified_date}).
    <br>© Copyright of game by Estudios Kremlinois (developer) and RockGame S.A. (publisher).
    <br>© Copyright of this tool (but not game data) by Peter Hozák under MIT License,
      <a href="https://github.com/Aprillion/urbek">https://github.com/Aprillion/urbek</a>.

    <p><b>How to read the table:</b>
    <dl>
      <dt>Village house needs 15 squares with 6 residents or more (on each square) within radius 3
          (7x7 area with house in the centre), and produces 8 residents:
          <dd>15□  6 ⊙3 <span class="plus">+8</span>
      <dt>Villa needs 25 residents or less within radius 4, and produces 6 residents per square
          (24 in total):
          <dd><span class="less">≤25 ⊙4 <span class="plus">+6</span></span>:
      <dt>Minimarket consumes 200 Food:
          <dd><span class="minus">-200</span>
      <dt>Lumbercamp extracts 10 natural resources from trees within radius 5, and produces 15 wood:
          <dd><span class="extracted">-10 ⊙5</span> <span class="plus">+15</span>
      <dt>Rebel house needs that no other Rebel houses and no Police stations are within radius 5:
          <dd>['<span class="less">0 rebelde ⊙5</span>', '<span class="less">0 pacos ⊙5</span>']
    </dl>
    <br>
    <p>Use <b>SHIFT+scroll</b> for horizontal scrolling on desktop.
      On mobile, you can tap a row to highlight it before scrolling (and/or rotate screen).
    <p>
      <label>Select game language: <select id="language" autocomplete="off">{option(languages)}</select></label>
  </div>
  <table>
    <thead>
      <tr>{th(columns)}</tr>
    </thead>
    {tr(td(b) for b in buildings)}
  </table>""" + """
  <script>
    function changeLanguage(ev) {
      const lang = ev.target.value
      for (el of document.querySelectorAll(".translate")) {
        try {
          const text = JSON.parse(el.getAttribute("title"))[lang]
          el.innerText = text
        } catch (e) {
          console.error(el, e)
        }
      }
    }
    document.getElementById("language").addEventListener("change", changeLanguage)
  </script>
</body>
</html>
"""
fname = 'docs/index.html'
with open(fname, 'wb') as f:
    f.write(html.encode("utf-8"))

# use `npx live-server docs` for live preview of the output